<a href="https://colab.research.google.com/github/robbarto2/Generative_AI_Course/blob/main/IMDB_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated this line
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


# Define the custom callback with a check for the 'loss' key
class LossLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'loss' in logs:
            print(f"Epoch: {state.epoch}, Loss: {logs['loss']:.4f}")

# Initialize the Trainer with the callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    callbacks=[LossLoggerCallback()]  # Add the loss logger callback here
)

# Fine-tune the modeltrainer.train()
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.210000,0.199369
2,0.127100,0.203933
3,0.066100,0.268391


Epoch: 0.3198976327575176, Loss: 0.3028
Epoch: 0.6397952655150352, Loss: 0.2361
Epoch: 0.9596928982725528, Loss: 0.2100
Epoch: 1.2795905310300704, Loss: 0.1426
Epoch: 1.599488163787588, Loss: 0.1259
Epoch: 1.9193857965451055, Loss: 0.1271
Epoch: 2.239283429302623, Loss: 0.0840
Epoch: 2.5591810620601407, Loss: 0.0653
Epoch: 2.8790786948176583, Loss: 0.0661


TrainOutput(global_step=4689, training_loss=0.1473099270668282, metrics={'train_runtime': 2070.079, 'train_samples_per_second': 36.231, 'train_steps_per_second': 2.265, 'total_flos': 1.9733329152e+16, 'train_loss': 0.1473099270668282, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.268391489982605, 'eval_runtime': 179.416, 'eval_samples_per_second': 139.341, 'eval_steps_per_second': 17.418, 'epoch': 3.0}


In [ ]:
# Calculate the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())

# Calculate the number of trainable parameters (those that require gradients)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Calculate the number of frozen parameters (those that do not require gradients)
frozen_params = total_params - trainable_params

# Print the results
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Frozen parameters: {frozen_params}")


Total parameters: 109483778
Trainable parameters: 109483778
Frozen parameters: 0


Inference

Batch Text Analysis

In [ ]:
import torch

texts = [
    "The special effects were amazing and the plot was thrilling!",
    "The plot was confusing and the characters were dull.",
    "An excellent movie that I would recommend to everyone.",
    "I think this movie is a pass"
]

# Tokenize the input
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move inputs to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = inputs.to(device) # Move input tensors to the same device as the model

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probs = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted classes
predicted_classes = torch.argmax(probs, dim=-1)

# Define the label map - replace with your actual labels
label_map = {0: "negative", 1: "positive"}

# Convert class indices to labels
predicted_labels = [label_map[class_idx.item()] for class_idx in predicted_classes]

# Print results
for text, label, prob in zip(texts, predicted_labels, probs.max(dim=-1).values):
    print(f"Text: {text}\nPrediction: {label}, Probability: {prob.item():.4f}\n")

Text: The special effects were amazing and the plot was thrilling!
Prediction: positive, Probability: 0.9974

Text: The plot was confusing and the characters were dull.
Prediction: negative, Probability: 0.9982

Text: An excellent movie that I would recommend to everyone.
Prediction: positive, Probability: 0.9984

Text: I think this movie is a pass
Prediction: negative, Probability: 0.9925



Save the model and tokenizer

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Create a directory on MyDrive and copy the mode results